In [26]:
import datetime
import hashlib
import re
import time
from pathlib import Path
from collections import defaultdict

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


# =========================
# CONFIG
# =========================
LEAGUE_ID = 8
SPORT_ID = 9

MIN_SEASON = "2016-17"
HEADLESS = False
WAIT = 30

# Varsity is often 12; keep a couple fallbacks
LEVEL_ID_CANDIDATES = [12, 11, 10, 13]

OUTPUT_DIR = Path(r"C:\Users\User\OneDrive - 2020 Companies\Desktop\IHS Soccer\Boys\2026")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DEBUG_DIR = OUTPUT_DIR / "debug"
DEBUG_DIR.mkdir(parents=True, exist_ok=True)

TEAM_NAME_CORRECTIONS = {
    "Lakeside": "Lakeside (Seattle)",
    "Lakeside (Sea)": "Lakeside (Seattle)",
    "Seattle Prep.": "Seattle Prep",
}

ORDINAL_SUFFIX_RE = re.compile(r"(\d+)(st|nd|rd|th)\b", re.IGNORECASE)


# =========================
# Driver
# =========================
def build_driver(headless: bool) -> webdriver.Chrome:
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    # Helps on JS-heavy pages sometimes
    opts.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=opts)
    driver.set_page_load_timeout(60)
    return driver


# =========================
# Season generation (no dropdown needed)
# =========================
def current_school_year(today: datetime.date | None = None) -> str:
    today = today or datetime.date.today()
    # rollover ~ July (good enough for school-year inference)
    start = today.year if today.month >= 7 else today.year - 1
    return f"{start}-{str((start + 1) % 100).zfill(2)}"


def generate_school_years(min_season: str) -> list[str]:
    start_year = int(min_season.split("-")[0])
    end_year = int(current_school_year().split("-")[0])
    return [f"{y}-{str((y + 1) % 100).zfill(2)}" for y in range(start_year, end_year + 1)]


def season_urls() -> list[tuple[str, int, str]]:
    urls = []
    for y in generate_school_years(MIN_SEASON):
        for level_id in LEVEL_ID_CANDIDATES:
            url = (
                f"https://www.metroleaguewa.org/sport/?leagueid={LEAGUE_ID}"
                f"&level_id={level_id}&school_year={y}&sportid={SPORT_ID}"
            )
            urls.append((y, level_id, url))
    # ensure earliest season first, and level_id order per season
    urls.sort(key=lambda x: (int(x[0].split("-")[0]), LEVEL_ID_CANDIDATES.index(x[1])))
    return urls


# =========================
# Normalization / IDs
# =========================
def normalize_team_name(name: str) -> str:
    if not isinstance(name, str):
        return name
    name = name.strip()
    return TEAM_NAME_CORRECTIONS.get(name, name)


def generate_game_id(season: str, date_raw: str, time_raw: str, home: str, away: str) -> str:
    s = f"{season}_{date_raw}_{time_raw}_{home}_{away}"
    return hashlib.md5(s.encode("utf-8")).hexdigest()


# =========================
# Date parsing
# =========================
def season_years(season: str) -> tuple[int, int]:
    a, b = season.split("-")
    start = int(a)
    end = int(b)
    if end < 100:
        end += 2000
    return start, end


def infer_year_from_season_and_month(season: str, month: int, rollover_month: int = 7) -> int:
    start, end = season_years(season)
    return start if month >= rollover_month else end


def parse_match_datetime(season: str, date_heading: str, time_str: str) -> pd.Timestamp:
    if not isinstance(date_heading, str) or not date_heading.strip():
        return pd.NaT

    d = date_heading.strip()
    d = ORDINAL_SUFFIX_RE.sub(r"\1", d)
    d = re.sub(r"^\w+,\s*", "", d)  # remove weekday prefix like "Saturday,"
    d = re.sub(r"\s+", " ", d)

    t = (time_str or "").strip()
    if not t or t.upper() == "TBD":
        t = "12:00 PM"

    # Try if year is present (sometimes it is)
    dt = pd.to_datetime(f"{d} {t}", errors="coerce", infer_datetime_format=True)
    if pd.notna(dt) and dt.year > 1900:
        return dt

    d_only = pd.to_datetime(d, errors="coerce", infer_datetime_format=True)
    if pd.isna(d_only):
        return pd.NaT

    inferred_year = infer_year_from_season_and_month(season, int(d_only.month))
    return pd.to_datetime(
        f"{inferred_year}-{int(d_only.month):02d}-{int(d_only.day):02d} {t}",
        errors="coerce",
    )


# =========================
# Context helpers (iframe-safe)
# =========================
def _has_any(driver, css: str) -> bool:
    return len(driver.find_elements(By.CSS_SELECTOR, css)) > 0


def switch_to_context_with(driver, css_list: list[str], timeout: int = WAIT) -> bool:
    """
    Tries default content + iframes until one contains any selector in css_list.
    Leaves driver focused in the matching context if found.
    """
    end = time.time() + timeout

    def found_here() -> bool:
        return any(_has_any(driver, css) for css in css_list)

    while time.time() < end:
        driver.switch_to.default_content()
        if found_here():
            return True

        frames = driver.find_elements(By.TAG_NAME, "iframe")
        for f in frames:
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame(f)
                if found_here():
                    return True

                # one nested level (just in case)
                nested = driver.find_elements(By.TAG_NAME, "iframe")
                for nf in nested:
                    try:
                        driver.switch_to.frame(nf)
                        if found_here():
                            return True
                    except Exception:
                        pass
                    finally:
                        try:
                            driver.switch_to.parent_frame()
                        except Exception:
                            driver.switch_to.default_content()
            except Exception:
                continue

        time.sleep(0.4)

    driver.switch_to.default_content()
    return False


def save_debug(driver, season: str, level_id: int, tag: str):
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    base = DEBUG_DIR / f"{season}_level{level_id}_{tag}_{ts}"
    try:
        (base.with_suffix(".html")).write_text(driver.page_source, encoding="utf-8", errors="ignore")
    except Exception:
        pass
    try:
        driver.save_screenshot(str(base.with_suffix(".png")))
    except Exception:
        pass


# =========================
# Full season filter
# =========================
def set_full_season_filter(driver) -> bool:
    """
    Uses the confirmed control:
      <select id="filter_date_range_kword"> ... value="season" ...
    Works in whichever context (default or iframe) contains it.
    """
    ok = switch_to_context_with(driver, ["#filter_date_range_kword"], timeout=10)
    if not ok:
        return False

    try:
        sel_el = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "filter_date_range_kword")))
        sel = Select(sel_el)
        cur = (sel.first_selected_option.get_attribute("value") or "").strip().lower()
        if cur != "season":
            # staleness wait on some existing schedule node if present
            old = None
            for css in [".schedule_contents", ".event_row", ".schedule_date_heading"]:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    old = els[0]
                    break
            sel.select_by_value("season")
            if old is not None:
                try:
                    WebDriverWait(driver, 10).until(EC.staleness_of(old))
                except Exception:
                    pass
        return True
    except Exception:
        return False


def load_lazy(driver, rounds: int = 12):
    """
    If full season still lazy-loads, scrolling often forces additional dates to render.
    """
    last_rows = -1
    for _ in range(rounds):
        rows = driver.find_elements(By.CSS_SELECTOR, ".event_row")
        if len(rows) == last_rows:
            break
        last_rows = len(rows)
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except Exception:
            pass
        time.sleep(0.7)
    try:
        driver.execute_script("window.scrollTo(0, 0);")
    except Exception:
        pass
    time.sleep(0.3)


# =========================
# Scrape season with correct per-row dates
# =========================
def scrape_season(driver, season: str, level_id: int, url: str) -> list[dict]:
    driver.switch_to.default_content()
    driver.get(url)
    WebDriverWait(driver, WAIT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(1.0)

    # Ensure we're in the context that contains the schedule content
    ok = switch_to_context_with(driver, [".schedule_date_heading", ".event_row", ".schedule_contents", "#filter_date_range_kword"], timeout=WAIT)
    if not ok:
        save_debug(driver, season, level_id, "no_schedule_context")
        return []

    # Force Full Season if possible
    set_full_season_filter(driver)

    # Re-locate schedule context (it may rerender)
    switch_to_context_with(driver, [".schedule_date_heading", ".event_row", ".schedule_contents"], timeout=15)

    # Let rows populate
    load_lazy(driver)

    rows = driver.find_elements(By.CSS_SELECTOR, ".event_row")
    if not rows:
        save_debug(driver, season, level_id, "no_rows")
        return []

    matches = []
    row_index = 0
    current_date = None

    # Best method: walk headings + rows in DOM order
    items = driver.find_elements(By.XPATH, "//*[contains(@class,'schedule_date_heading') or contains(@class,'event_row')]")

    for el in items:
        cls = (el.get_attribute("class") or "").lower()

        if "schedule_date_heading" in cls:
            current_date = (el.text or "").strip()
            continue

        if "event_row" in cls:
            # Fallback if DOM order doesn't include headings properly:
            date_raw = current_date
            if not date_raw:
                try:
                    dh = el.find_element(By.XPATH, "preceding::*[contains(@class,'schedule_date_heading')][1]")
                    date_raw = (dh.text or "").strip()
                except Exception:
                    date_raw = ""

            if not date_raw:
                continue

            # time
            try:
                time_raw = (el.find_element(By.CLASS_NAME, "event_time").text or "").strip()
            except Exception:
                time_raw = ""

            # game type
            try:
                game_type = (el.find_element(By.CLASS_NAME, "game_type").text or "").strip()
            except Exception:
                game_type = ""

            if game_type in ["Scrimmage", "Jamboree"]:
                continue

            # teams/scores
            try:
                away_team = (el.find_element(By.CSS_SELECTOR, ".event_team .event_team_name").text or "").strip()
                away_score = (el.find_element(By.CSS_SELECTOR, ".event_team .event_team_score").text or "").strip()
                home_team = (el.find_element(By.CSS_SELECTOR, ".event_team_home .event_team_name").text or "").strip()
                home_score = (el.find_element(By.CSS_SELECTOR, ".event_team_home .event_team_score").text or "").strip()
            except Exception:
                continue

            away_team = normalize_team_name(away_team)
            home_team = normalize_team_name(home_team)

            matches.append({
                "Season": season,
                "LevelID": level_id,
                "DateRaw": date_raw,
                "TimeRaw": time_raw,
                "Home Team": home_team,
                "Away Team": away_team,
                "Home Score": home_score,
                "Away Score": away_score,
                "Game Type": game_type,
                "RowIndex": row_index,  # page order tie-breaker
                "GameID": generate_game_id(season, date_raw, time_raw, home_team, away_team),
            })
            row_index += 1

    return matches


# =========================
# Pipeline
# =========================
def build_played_order(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["MatchDateTime"] = [
        parse_match_datetime(s, d, t)
        for s, d, t in zip(out["Season"], out["DateRaw"], out["TimeRaw"])
    ]
    out["DateISO"] = pd.to_datetime(out["MatchDateTime"], errors="coerce").dt.strftime("%Y-%m-%d")

    # strict, leakage-resistant ordering:
    # chronological, then stable DOM tie-break within season
    out["MatchDateTimeFill"] = out["MatchDateTime"].fillna(pd.Timestamp.max)
    out = out.sort_values(["Season", "MatchDateTimeFill", "RowIndex"], kind="mergesort").reset_index(drop=True)
    out["MatchOrder"] = range(1, len(out) + 1)
    return out.drop(columns=["MatchDateTimeFill"])


def main():
    driver = build_driver(HEADLESS)
    all_matches = []
    accepted_seasons = set()

    try:
        for season, level_id, url in season_urls():
            # accept first level_id that returns real rows for the season
            if season in accepted_seasons:
                continue

            print(f"Scraping {season} → {url}")
            season_matches = scrape_season(driver, season, level_id, url)

            if season_matches:
                all_matches.extend(season_matches)
                accepted_seasons.add(season)
            else:
                # try next level_id for that same season
                continue

    finally:
        driver.quit()

    if not all_matches:
        raise RuntimeError(f"No matches scraped. Check debug folder: {DEBUG_DIR}")

    df = pd.DataFrame(all_matches).drop_duplicates(subset=["GameID"]).copy()

    df = build_played_order(df)

    out_csv = OUTPUT_DIR / "metroleague_results_all_games.csv"
    df.to_csv(out_csv, index=False)

    print("\nSaved:", out_csv)
    print("\nUnique DateRaw sample:")
    for d in pd.Series(df["DateRaw"]).dropna().drop_duplicates().head(20).to_list():
        print(" ", d)


if __name__ == "__main__":
    main()

Scraping 2016-17 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2016-17&sportid=9
Scraping 2017-18 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2017-18&sportid=9
Scraping 2018-19 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2018-19&sportid=9
Scraping 2019-20 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2019-20&sportid=9
Scraping 2020-21 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2020-21&sportid=9
Scraping 2021-22 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2021-22&sportid=9
Scraping 2022-23 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2022-23&sportid=9
Scraping 2023-24 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2023-24&sportid=9
Scraping 2024-25 → https://www.metroleaguewa.org/sport/?leagueid=8&level_id=12&school_year=2024-25&sportid=9
Scraping 2025-26 → 

C:\Users\User\AppData\Local\Temp\ipykernel_6236\412716844.py:138: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(f"{d} {t}", errors="coerce", infer_datetime_format=True)
C:\Users\User\AppData\Local\Temp\ipykernel_6236\412716844.py:142: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  d_only = pd.to_datetime(d, errors="coerce", infer_datetime_format=True)



Saved: C:\Users\User\OneDrive - 2020 Companies\Desktop\IHS Soccer\Boys\2026\metroleague_results_all_games.csv

Unique DateRaw sample:
  Thursday, Mar 9
  Friday, Mar 10
  Saturday, Mar 11
  Tuesday, Mar 14
  Wednesday, Mar 15
  Friday, Mar 17
  Monday, Mar 20
  Wednesday, Mar 22
  Friday, Mar 24
  Tuesday, Mar 28
  Thursday, Mar 30
  Friday, Mar 31
  Monday, Apr 3
  Wednesday, Apr 5
  Friday, Apr 7
  Monday, Apr 10
  Wednesday, Apr 12
  Monday, Apr 17
  Tuesday, Apr 18
  Wednesday, Apr 19
